# Udacity Project: Wrangle OSM Data to SQL- Houston, Texas 

## Introduction

For this project, I chose my hometown of Houston, Texas. I originally chose a small area containing Rice University and the museum district then moved my way to the larger metropolitan area. As the data covers a large area, I was unable to efficiently run my code on the full dataset and instead chose to sample the area using code provided by Udacity. I used the code provided and created during the Problem Set exercises as the basis for collecting and auditing the data. From there, I created the csv files and database (which proved to be one of the most difficult parts). I then used the sample project as a template for formulating questions for querying using sqlite. 

OSM query: 
https://www.openstreetmap.org/relation/2688911

Example project: 
https://gist.github.com/carlward/54ec1c91b62a5f911c42#file-sample_project-md


## Importing relevant libraries

In [11]:
#Note, several of these import statements appear later as well. This is only to keep track of relevant code 
import xml.etree.cElementTree as ET
import pprint
from collections import defaultdict
import re 
import csv
import codecs
import cerberus
import os
import pandas as pd 
import sqlite3
from pprint import pprint 

## Iterative Parsing to find all unique tag names

In [14]:
def count_tags(filename):
        osm_file = open(filename, "r")
        tag_types = defaultdict(int)
        for event, elem in ET.iterparse(osm_file):
            tag_type = elem.tag
            if tag_type not in tag_types.keys(): 
                tag_types[tag_type] = 1 
            else: tag_types[tag_type] += 1 
        return tag_types

def test():
    tags = count_tags('interpreter.osm')
    pprint(tags)

if __name__ == "__main__":
    test()

defaultdict(<type 'int'>, {'node': 278542, 'nd': 348101, 'member': 2515, 'tag': 188029, 'relation': 224, 'way': 41283, 'osm': 1})


## Audit Street names/Ways: Problematic characters

In [8]:
# Checking tags and K values for problematic characters 
"""
Uses 3 regular expressions to check for certain patterns in the tags. 
We would like to change the data from the "addr:street" type of keys to a dictionary like this:
{"address": {"street": "Some value"}}

Four tag categories in a dictionary:
  "lower", for tags that contain only lowercase letters and are valid,
  "lower_colon", for otherwise valid tags with a colon in their names,
  "problemchars", for tags with problematic characters, and
  "other", for other tags that do not fall into the other three categories.
"""
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

def key_type(element, keys):
    if element.tag == "tag":
        k = element.attrib['k']
        if re.search(lower, k): 
            keys["lower"] += 1 
        elif re.search(lower_colon, k): 
            keys["lower_colon"] += 1 
        elif re.search(problemchars, k): 
            keys["problemchars"] += 1
            print "problemchars = ", element.tag, element.attrib
        else: 
            keys["other"] += 1 
        pass
    return keys

def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)
    return keys

def test():
    keys = process_map('interpreter.osm')
    print(keys)

if __name__ == "__main__":
    test()

print "done"

{'problemchars': 0, 'lower': 93927, 'other': 3910, 'lower_colon': 90192}
done


# Improving Street Names

Most of the below code and comments were taken from the problem set prior to this project. I ran the code repeatedly to better understand the types of street names that would be caught and adjusted accordingly. Some streets had programatic fixes while others just needed a very specific one-off fix. While a bit tedious, this approach worked for this data set but may not have worked for a larger one. 

If needed, the code could be rewritten to search through each word in a street name rather than just the street type (which I believe was a suggestion on the Udacity forums). I thought it sufficient to look for incorrect/mispelled street types rather than the various ways one could write a street type. For my purposes, I saw no reason to go into the details of modifying and standardizing the highway names and types, especially with data that is put together by users and would likely have similar discrepancies in other cities' datasets. 

In [13]:
"""
- audit the OSMFILE and use the variable 'mapping' which reflects the changes needed 
    Note: not a generalized solution
- update_name: actually fixes the street name.
    The function takes a string with street name as an argument and should return the fixed name
"""

OSMFILE = "interpreter.osm"
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Circle", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons", "Plaza", "Speedway", "Staffordshire", "Way", "Park", 
            "Freeway", "Loop", "130", "Block", "Walk", "North", "East", "South", "West", "59", "I-10",
           "1960", "6", "Real", "Highway", "Trace"]

nums = [] #for use if we wanted to add street names that end with numbers to our expected list
#for i in range(200): 
#    nums.append(str(i))

mapping = { "St": "Street",
            "St.": "Street", 
            "Stree": "Street",
            "Ave": "Avenue",
            "Ave.": "Avenue", 
            "Rd.": "Road",
            "Rd": "Road",
            "Maroneal": "Maroneal Street",
            "Montrose": "Montrose Boulevard", 
            "Plaze": "Plaza",
            "Graustark": "Graustark Street",
            "Blvd": "Boulevard",
            "Blvd.": "Bouldevard", 
            "Ln": "Lane", 
            "Ln.": "Lane", 
            "Pkwy": "Parkway", 
            "Dr": "Drive", 
            "Dr.": "Drive", 
            "Ct": "Court", 
            "Sq": "Square", 
            "Sq.": "Square",
            "Lake": "Lane",
            "Bailey": "Bailey Street", 
            "Oaks": "Oaks Block",
            "Isle": "Isle Lane", 
            "Texas": "", 
            "Houston": "",
            "Mews": "Mews Plantation",
            "Cypress": "Cypress Road",
            "N": "North", 
           "N.": "North", 
           "E": "East", 
           "E.": "East", 
           "S": "South",
           "S.": "South",
           "W": "West",
           "W.": "West",
            "Ste": "Suite", 
            "Ste.": "Suite", 
           "Driscoll": "Driscoll Street",
           "Gulfcrest": "Gulfcrest Street",
           "Larchmont": "Larchmont Road", 
           "Durham": "Durham Drive", 
           "Welford": "Welford Drive"       
            }

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)

def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types

unmapped = set()

def update_name(name, mapping):
    street_type = street_type_re.search(name).group()
    if street_type not in expected: 
        if street_type in mapping:
            name = (name[:-len(street_type)] + mapping[street_type])
            print name 
        else: 
            #print "name not in mapping:", street_type            
            unmapped.add(name)
            #just in case we want to take a look and make sure nothing crazy is in the unmapped set        
    return name

def test():
    st_types = audit(OSMFILE)
    pprint(dict(st_types))
    for st_type, ways in st_types.iteritems():
        for name in ways:
            better_name = update_name(name, mapping)
            print name, "=>", better_name
            if name == "West Lexington St.":
                assert better_name == "West Lexington Street"
            if name == "Baldwin Rd.":
                assert better_name == "Baldwin Road"

if __name__ == '__main__':
    test()

{'1093': set(['F.M. 1093', 'Farm to Market 1093']),
 '1464': set(['FM 1464']),
 '1485': set(['FM 1485']),
 '1640': set(['FM 1640']),
 '2920': set(['FM 2920']),
 '362': set(['FM 362']),
 '529': set(['Farm To Market Rd 529']),
 '7A': set(['Fondren Road # 7A']),
 '90': set(['US 90']),
 'A': set(['Dallas St #A']),
 'Ave': set(['E Parkwood Ave']),
 'B': set(['Richmond Ave, Ste B']),
 'Bailey': set(['Bailey']),
 'Blvd': set(['Post Oak Blvd']),
 'Dr.': set(['Palomar Valley Dr.']),
 'Driscoll': set(['Driscoll']),
 'Durham': set(['Durham']),
 'E': set(['Avenue E', 'FM 646 Rd E', 'North Sam Houston Pkwy E']),
 'F': set(['San Felipe Road, suite F']),
 'Gulfcrest': set(['Gulfcrest']),
 'I-45': set(['I-45']),
 'Larchmont': set(['Larchmont']),
 'Rd': set(['Cypress North Houston Rd', 'Kuykendahl Rd', 'Rogerdale Rd']),
 'S': set(['Highway 6 S']),
 'S.': set(['Hwy 6 S.']),
 'St': set(['Canal St', 'Tuam St']),
 'St.': set(['10039 Bissonnet St.']),
 'St.,': set(['10039 Bissonnet St.,']),
 'Welford': set(

# Preparing for Database

## Creating a schema

In [15]:
# %load 'schema.py'
schema = {
    'node': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'lat': {'required': True, 'type': 'float', 'coerce': float},
            'lon': {'required': True, 'type': 'float', 'coerce': float},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'node_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    },
    'way': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'way_nodes': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'node_id': {'required': True, 'type': 'integer', 'coerce': int},
                'position': {'required': True, 'type': 'integer', 'coerce': int}
            }
        }
    },
    'way_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    }
}

## Gathering relevant data 

In [16]:
"""
The process for this transformation is as follows:
- Use iterparse to iteratively step through each top level element in the XML
- Shape each element into several data structures using a custom function
- Utilize a schema and validation library to ensure the transformed data is in the correct format
- Write each data structure to the appropriate .csv files
"""
import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET
import cerberus

OSM_PATH = "interpreter.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema

NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']


def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  

    if element.tag == 'node':
        for item in element.attrib:
            if item in NODE_FIELDS: 
                node_attribs[item] = element.attrib[item]
        for child in element: 
            nodes = {}
            k_problems = PROBLEMCHARS.search(child.attrib['k'])
            k_semi_colon = LOWER_COLON.search(child.attrib['k'])
            if k_problems:
                pass 
            elif k_semi_colon:
                m = k_semi_colon.group()
                k_value_split = m.split(":")
                if len(k_value_split) > 1:
                    nodes["id"]= element.attrib["id"]
                    nodes["key"] = "".join(k_value_split[1:]) 
                    nodes["value"] = child.attrib["v"]
                    nodes["type"] = k_value_split[0]
                    tags.append(nodes)
            else: 
                nodes["id"] = element.attrib["id"]
                nodes["key"] = child.attrib["k"]
                nodes["value"] = child.attrib["v"]
                nodes["type"] = "regular"
                tags.append(nodes)
        return {'node': node_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        for item in element.attrib:
            if item in WAY_FIELDS:
                way_attribs[item] = element.attrib[item]
        pos = 0 
        for child in element: 
            way_n = {} 
            way_t = {} 
            
            if child.tag == "nd": 
                problems = PROBLEMCHARS.search(child.attrib["ref"]) 
                if problems: 
                    pass 
                else: 
                    way_n["id"] = element.attrib["id"]
                    way_n["node_id"] = child.attrib["ref"]
                    way_n["position"] = pos
                    way_nodes.append(way_n)
                    pos += 1
                
            if child.tag == "tag":
                problems = PROBLEMCHARS.search(child.attrib["k"]) 
                colattribs = LOWER_COLON.search(child.attrib["k"])
                
                if problems:
                    pass 
                elif colattribs: 
                    m = colattribs.group()
                    splits = m.split(":")
                    if len(splits) >1: 
                        way_t["id"] = element.attrib["id"]
                        way_t["key"] = "".join(splits[1:])
                        way_t["value"] = child.attrib["v"]
                        way_t["type"] = splits[0]
                        tags.append(way_t)
                else: 
                    way_t["id"] = element.attrib["id"]
                    way_t["key"] = child.attrib["k"]
                    way_t["value"] = child.attrib["v"]
                    way_t["type"] = "regular"
                    tags.append(way_t)
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}


# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""
    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""

    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))

class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])

if __name__ == '__main__':
    process_map(OSM_PATH, validate=True)

print "Code finished running"

Code finished running


# Create the database

In [17]:
#$ sqlite3 create database;
#connect to database - already created
sqlite_file = 'interpreter.db'
conn = sqlite3.connect(sqlite_file)
cur = conn.cursor()

cur.execute('''DROP TABLE IF EXISTS nodes''')
conn.commit()
cur.execute('''DROP TABLE IF EXISTS nodetags''')
conn.commit()
cur.execute('''DROP TABLE IF EXISTS ways''')
conn.commit()
cur.execute('''DROP TABLE IF EXISTS waytags''')
conn.commit()
cur.execute('''DROP TABLE IF EXISTS waynodes''')
conn.commit()

cur.execute('''
create table nodes 
(
    nodeid integer primary key not null, 
    lat real, 
    lon real, 
    user text, 
    uid integer,
    version integer,
    changeset integer,
    timestamp text  
)''')
conn.commit()

cur.execute('''
create table nodetags
(
    id integer,
    key text, 
    value text, 
    type text, 
    foreign key (id) references nodes (nodeid)

)''')
conn.commit()
            
#id above is the node id and is also the primary key for the nodes table 
cur.execute('''
create table ways
(
    wayid integer primary key not null, 
    user text, 
    uid integer, 
    version text, 
    changeset integer,
    timestamp text
)''')
conn.commit()

cur.execute('''

create table waynodes
(
    wayid integer not null, 
    node_id integer not null, 
    position integer not null, 
    foreign key (wayid) references ways (wayid)
    foreign key (node_id) references nodes (nodeid)

)''')
conn.commit()             
#id above is a way id and different from a node id
 
cur.execute('''
    
create table waytags 
(
    wayid integer not null, 
    key text not null, 
    value text not null, 
    type text not null, 
    foreign key (wayid) references ways (wayid)

)''')
conn.commit()

print "code finished"

code finished


## Read the data into the tables

In [18]:
# Read in the csv file as a dictionary, format the data as a list of tuples:

with open('nodes.csv','rb') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['id'], i['lat'],i['lon'], i['user'].decode("utf-8"), i['uid'], i['version'],i['changeset'], i['timestamp']) for i in dr]
# insert the formatted data
cur.executemany("INSERT INTO nodes(nodeid, lat, lon , user, uid, version, changeset, timestamp) VALUES (?, ?, ?, ?, ?, ?, ?, ?);", to_db)
# commit the changes
conn.commit()
 
with open('nodes_tags.csv','rb') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['id'], i['key'].decode("utf-8"),i['value'].decode("utf-8"), i['type']) for i in dr]
cur.executemany("INSERT INTO nodetags(id, key, value, type) VALUES (?, ?, ?, ?);", to_db)
conn.commit()

with open('ways.csv','rb') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['id'], i['user'].decode("utf-8"),i['uid'], i['version'], i['changeset'], i['timestamp']) for i in dr]
cur.executemany("INSERT INTO ways(wayid, user, uid , version, changeset, timestamp) VALUES (?, ?, ?, ?, ?, ?);", to_db)
conn.commit()

with open('ways_nodes.csv','rb') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['id'], i['node_id'],i['position']) for i in dr]
cur.executemany("INSERT INTO waynodes(wayid, node_id, position) VALUES (?, ?, ?);", to_db)
conn.commit()

with open('ways_tags.csv','rb') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['id'], i['key'].decode("utf-8"),i['value'].decode("utf-8"), i['type']) for i in dr]
cur.executemany("INSERT INTO waytags(wayid, key, value, type) VALUES (?, ?, ?, ?);", to_db)
conn.commit()

print "Code finished"

Code finished


# Data Overview

## Check file size

In [19]:
# File size 
import os
print "Original Data: ", os.path.getsize('/Users/irasema/Desktop/DataScience/Udacity/Data Analyst/Project 2/interpreter.osm')
print "Nodes CSV: ", os.path.getsize('/Users/irasema/Desktop/DataScience/Udacity/Data Analyst/Project 2/nodes.csv')
print "Nodes Tags CSV: ", os.path.getsize('/Users/irasema/Desktop/DataScience/Udacity/Data Analyst/Project 2/nodes_tags.csv')
print "Ways CSV: ", os.path.getsize('/Users/irasema/Desktop/DataScience/Udacity/Data Analyst/Project 2/ways.csv')
print "Ways Tags CSV: ", os.path.getsize('/Users/irasema/Desktop/DataScience/Udacity/Data Analyst/Project 2/ways_tags.csv')
print "Ways Nodes CSV: ", os.path.getsize('/Users/irasema/Desktop/DataScience/Udacity/Data Analyst/Project 2/ways_nodes.csv')
print "Database: ", os.path.getsize('/Users/irasema/Desktop/DataScience/Udacity/Data Analyst/Project 2/interpreter.db')

#Resource:
#https://stackoverflow.com/questions/6591931/getting-file-size-in-python
#https://docs.python.org/3/library/os.path.html#os.path.getsize

Original Data:  66749940
Nodes CSV:  23374752
Nodes Tags CSV:  564723
Ways CSV:  2452684
Ways Tags CSV:  6043017
Ways Nodes CSV:  8296465
Database:  35467264


## Are all data coordinates within the limits of the original query? Is anything out of place?

In [20]:
query = ''' select max(lat), min(lat), max(lon), min(lon) from nodes '''
db = sqlite3.connect("interpreter.db")
cursor = db.cursor()
cursor.execute(query)
rows = cursor.fetchall()
df = pd.DataFrame(rows)
print df

         0          1         2          3
0  30.1831  29.463502 -94.82301 -96.100149


The coordinates seem to all fall within the Houston metropolitan area. 

## How many Nodes are there?

In [21]:
cursor = db.cursor()
query = "select count(*) from nodes"
cursor.execute(query)
rows = cursor.fetchall()
df = pd.DataFrame(rows)
print df

        0
0  278542


## How many ways?

In [22]:
query = "Select count(*) from ways"
cursor.execute(query)
rows = cursor.fetchall()
df = pd.DataFrame(rows)
print df

       0
0  41283


## Node tags?

In [23]:
query = "Select count(*) from nodetags"
cursor.execute(query)
rows = cursor.fetchall()
df = pd.DataFrame(rows)
print df

       0
0  15045


## Way tags?

In [24]:
query = "Select count(*) from waytags"
cursor.execute(query)
rows = cursor.fetchall()
df = pd.DataFrame(rows)
print df

        0
0  172130


## What are the most common node tags?

In [25]:
query = ''' select key, value, count(*) as count from nodetags 
group by key, value order by count desc limit 20 
'''
cursor.execute(query)
rows = cursor.fetchall()
df = pd.DataFrame(rows)
print df


            0                  1     2
0     highway     turning_circle  2313
1       power              tower  1379
2       power               pole   932
3     highway    traffic_signals   751
4     highway           crossing   358
5     highway       turning_loop   331
6     railway     level_crossing   321
7    state_id                 48   273
8   county_id                201   225
9       state                 TX   219
10    natural               tree   205
11    amenity   place_of_worship   193
12   religion          christian   188
13       city            Houston   177
14    barrier               gate   163
15     noexit                yes   135
16   building              house   127
17    highway  motorway_junction   113
18    created         12/08/2003   112
19   crossing              zebra    91


## What are the most common way tags?

In [26]:
query = """Select key, value, count(*) as count from waytags group by key, value 
order by count desc limit 20 
"""
cursor.execute(query)
rows = cursor.fetchall()
df = pd.DataFrame(rows)
print df

              0                                                  1      2
0          cfcc                                                A41  11581
1      reviewed                                                 no  10782
2        county                                         Harris, TX  10533
3       highway                                        residential   9567
4       highway                                            service   8550
5      building                                                yes   7425
6        oneway                                                yes   4936
7     separated                                                 no   4376
8        source                     tiger_import_dch_v0.6_20070830   3779
9      building                                              house   2857
10      service                                           driveway   2425
11    name_type                                                 Dr   2395
12      highway                       

## What about across way and node tags? 

In [27]:
query = ''' select key, value, count(*) as count
from (select * from waytags union all select * from nodetags) as subq
group by key, value order by count desc limit 40 
'''
cursor.execute(query)
rows = cursor.fetchall()
df = pd.DataFrame(rows)
print df

              0                                                  1      2
0          cfcc                                                A41  11581
1      reviewed                                                 no  10840
2        county                                         Harris, TX  10533
3       highway                                        residential   9567
4       highway                                            service   8550
5      building                                                yes   7457
6        oneway                                                yes   4936
7     separated                                                 no   4376
8        source                     tiger_import_dch_v0.6_20070830   3779
9      building                                              house   2984
10      service                                           driveway   2425
11    name_type                                                 Dr   2395
12      highway                       

## Why are there so many zebra crossing tags?

In [30]:
query = "Select id, key, value, type from nodetags where value = 'zebra'"
cursor.execute(query)
rows = cursor.fetchall()
df = pd.DataFrame(rows)
print df

             0             1      2        3
0    152566394      crossing  zebra  regular
1    152595847      crossing  zebra  regular
2   2595305241      crossing  zebra  regular
3   2920452327      crossing  zebra  regular
4   2982750738      crossing  zebra  regular
5   2982750748      crossing  zebra  regular
6   3121179043      crossing  zebra  regular
7   3132361363      crossing  zebra  regular
8   3132361384      crossing  zebra  regular
9   3220290864      crossing  zebra  regular
10  3242607179      crossing  zebra  regular
11  3253105978  crossing_ref  zebra  regular
12  3568506288      crossing  zebra  regular
13  3568506378      crossing  zebra  regular
14  3580613997      crossing  zebra  regular
15  3662128893      crossing  zebra  regular
16  3723389250      crossing  zebra  regular
17  3872980273      crossing  zebra  regular
18  4020665615      crossing  zebra  regular
19  4247739958      crossing  zebra  regular
20  4247757690      crossing  zebra  regular
21  424775

In [31]:
#Combining tables to get additional information on Zebra tags
#Filters for node tags involving the 'zebra' value
query = '''Select nodes.lat, nodes.lon, nodeid, nodetags.key, nodetags.value, nodetags.type, 
waytags.wayid, waytags.key, waytags.value, waytags.type
from nodes, nodetags, ways, waytags, waynodes where nodes.nodeid = nodetags.id 
and waynodes.wayid = waytags.wayid and ways.wayid = waynodes.wayid 
and waynodes.node_id = nodes.nodeid 
and nodetags.value = 'zebra' 
'''
cursor.execute(query)
rows = cursor.fetchall()
df = pd.DataFrame(rows)
print df

            0          1           2             3      4        5          6  \
0   30.048567 -95.151942  3568506288      crossing  zebra  regular   15443194   
1   30.048567 -95.151942  3568506288      crossing  zebra  regular   15443194   
2   30.048567 -95.151942  3568506288      crossing  zebra  regular   15443194   
3   30.048567 -95.151942  3568506288      crossing  zebra  regular   15443194   
4   30.048567 -95.151942  3568506288      crossing  zebra  regular   15443194   
5   30.048567 -95.151942  3568506288      crossing  zebra  regular   15443194   
6   30.048567 -95.151942  3568506288      crossing  zebra  regular   15443194   
7   30.048567 -95.151942  3568506288      crossing  zebra  regular   15443194   
8   30.048567 -95.151942  3568506288      crossing  zebra  regular   15443194   
9   30.059441 -95.244726  4274821477      crossing  zebra  regular   21863953   
10  30.059441 -95.244726  4274821477      crossing  zebra  regular   21863953   
11  30.059441 -95.244726  42

Upon further exploration, (looking at key/value pairs, googling the lat/lon, and googling the phrase 'zebra crossings osm') it seems the zebra crossing is a type of crosswalk (nonspecific to zebras or painted as zebras for the Houston Zoo). 

This may be an example of not when understanding the data leads to findings that may seem off but after some digging, actually make sense. I figured though the result was not noteable and could wholely be taken out of the report, the investigative procedure is helpful to demonstrate. 

https://wiki.openstreetmap.org/wiki/Approved_features/Road_crossings

# Data Exploration continued

## Most common postcodes

In [50]:
query = ''' select value, count(*) as count from nodetags 
where key = 'postcode' 
group by value 
order by count desc
limit 10
'''
cursor.execute(query)
rows = cursor.fetchall()
df = pd.DataFrame(rows)
print df

       0   1
0  77096  51
1  77339  18
2  77401  18
3  77002   8
4  77076   7
5  77586   7
6  77006   6
7  77098   5
8  77027   4
9  77085   4


The Houston Metropolitan area is fairly large and covers a range of postcodes. After some searching, I determined that most of the postcodes above actually fall within the area. However, the postcode 88581 actually belongs to El Paso, Texas.

Resources: 
https://data.mongabay.com/igapo/zip_codes/metropolitan-areas/metro-alpha/Houston%20(TX)1.html


## Users 

In [33]:
# This query combines the nodes and ways table to find all users who have contributed to either
query = ''' select distinct(subq.user), uid
from (select uid, user from nodes union all select uid, user from ways) subq
'''
cursor.execute(query)
rows = cursor.fetchall()
df = pd.DataFrame(rows)
print df

                    0        1
0           brianboru     9065
1           davidearl     3582
2     woodpeck_repair   145231
3           andrewpmk     1679
4     woodpeck_fixbot   147510
5             scottyc   496606
6            afdreher  1110270
7              clay_c   119881
8       RoadGeek_MD99   475877
9             Memoire  2176227
10        NayanataraM  2053369
11             choess   396743
12           Sundance   239998
13           karitotp  2748195
14           KristenK  1494110
15            cammace  3119079
16          Bootprint    76077
17          Bellhalla   219655
18        maxerickson   360392
19          EiderDuck  1924872
20            skquinn   243003
21            ridixcr  2508151
22             chewey  1208453
23              MikeN   135163
24           Iowa Kid   703517
25            wikewag  5319602
26                NE2   207745
27            ELadner    22925
28        gridlockjoe    70659
29            dufekin    30521
...               ...      ...
1363    

## Top 10 contributors? 

In [34]:
# top 10 contributing users across the nodes and ways tables 
query = ''' select uid, subq.user, count(*) as count
from 
(select uid, user from nodes union all select uid, user from ways) as subq
group by subq.user
order by count desc
limit 10
'''
cursor.execute(query)
rows = cursor.fetchall()
df = pd.DataFrame(rows)
print df

         0                1      2
0  1110270         afdreher  50264
1   147510  woodpeck_fixbot  35051
2   496606          scottyc  19328
3  3119079          cammace  19259
4   119881           clay_c  16125
5     9065        brianboru  11224
6   243003          skquinn   8680
7   475877    RoadGeek_MD99   7622
8   672878         TexasNHD   7005
9  2176227          Memoire   6540


## Users with 1 post

In [35]:
query = ''' 
select count (*) from 
(select uid, user, count(*) as counts
from (select uid, user from nodes union all select uid, user from ways) as subq
group by subq.user
having counts = 1
order by counts desc ) as substuff 
'''
cursor.execute(query)
rows = cursor.fetchall()
df = pd.DataFrame(rows)
print df


     0
0  265


## Restaurants 

In [36]:
# Note, this query only draws from nodetags (not waytags - to be addressed later) 
query = ''' select names.name, value, count (*) as count 
from nodetags, 
(SELECT distinct(id) as restid FROM nodetags WHERE value= 'restaurant' or value= 'fast_food') 
as rest, 
(select value as name, id as nameid from nodetags where key = 'name') as names 

where nodetags.id = rest.restid 
and nodetags.id = names.nameid 
and nodetags.key = 'cuisine' 

group by name 
order by count desc 
'''
cursor.execute(query)
rows = cursor.fetchall()
df = pd.DataFrame(rows)
print df

                                            0                     1   2
0                                      Subway              sandwich  11
1                                     Wendy's                burger   7
2                             Jack in the Box                burger   4
3                                  McDonald's                burger   4
4                                 Chick-fil-A               chicken   3
5                           Schlotzsky's Deli              sandwich   3
6                                 Whataburger                burger   3
7                                 Burger King                burger   2
8                                 Jamba Juice                drinks   2
9                                         KFC               chicken   2
10                            Little Caesar's                 pizza   2
11                              Panda Express               chinese   2
12                                   Popeye's               chic

## Most prevalent cuisines?

In [37]:
# Overall, what cuisines are most prevalent? 

query = ''' select value, count (*) as count
from nodetags, 
(SELECT distinct(id) as restid FROM nodetags WHERE value= 'restaurant' or value = 'fast_food') as rest, 
(select value as name, id as nameid from nodetags where key = 'name') as names 

where nodetags.id = rest.restid 
and nodetags.id = names.nameid 
and nodetags.key = 'cuisine' 

group by value 
order by count desc
'''

cursor.execute(query)
rows = cursor.fetchall()
df = pd.DataFrame(rows)
print df

                       0   1
0                 burger  26
1               sandwich  16
2                  pizza  13
3                mexican  12
4                chicken   9
5                italian   4
6              ice_cream   3
7               american   2
8               barbecue   2
9                chinese   2
10                drinks   2
11                indian   2
12        latin_american   2
13                 wings   2
14        Oaxaca;mexican   1
15                 asian   1
16             breakfast   1
17                 cajun   1
18                 greek   1
19              japanese   1
20          local;burger   1
21         mediterranean   1
22  mediterranean;buffet   1
23                   pie   1
24                 sushi   1
25                  thai   1
26                 vegan   1
27            vietnamese   1


In [38]:
# What are the most popular places and their corresponding cuisine? 
# Note: This query draws from waytags not nodetags 

query = ''' select names.name, value, count (*) as count 
from waytags, 
(SELECT distinct(wayid) as restid FROM waytags WHERE value= 'restaurant' or value= 'fast_food') 
as rest, 
(select value as name, wayid as nameid from waytags where key = 'name') as names 

where waytags.wayid = rest.restid 
and waytags.wayid = names.nameid 
and waytags.key = 'cuisine' 

group by name 
order by count desc 
'''
cursor.execute(query)
rows = cursor.fetchall()
df = pd.DataFrame(rows)
print df

                               0                1   2
0                    Whataburger           burger  13
1                     McDonald's           burger   4
2                    Burger King           burger   3
3                    Chick-fil-A          chicken   3
4                Jack in the Box           burger   2
5                  Panda Express          chinese   2
6                    Taco Cabana          mexican   2
7                 Baskin-Robbins        ice_cream   1
8                 Breakfast Klub  diner;breakfast   1
9                      Carl's Jr           burger   1
10                      Chipotle          mexican   1
11                   Fudrucker's           burger   1
12                Goodson's Cafe         american   1
13          Hartz Chicken Buffet          chicken   1
14               Jack In The Box           burger   1
15                     Los Cucos          mexican   1
16  Los Cucos Mexican Restaurant           texmex   1
17                  Olive Ga

## Fast food information? 

In [39]:
# This query draws from the node_ids of fast_food restaurants and gives 
# the additional information that are stored in related nodetags
query = ''' 
SELECT key, value from nodetags, 
(select distinct(id) from nodetags where value = 'fast_food') as fast 
where nodetags.id = fast.id 
'''
cursor.execute(query)
rows = cursor.fetchall()
df = pd.DataFrame(rows)
print df

                 0                          1
0          amenity                  fast_food
1          cuisine                     burger
2             name                Burger King
3          amenity                  fast_food
4          cuisine                    chicken
5             name     Popeye's Fried Chicken
6          website    http://www.popeyes.com/
7          amenity                  fast_food
8             name            Jack in the Box
9             city                    Tomball
10           state                         TX
11         amenity                  fast_food
12           is_in                Tomball, Tx
13            name                  Pizza Hut
14         amenity                  fast_food
15         cuisine                     burger
16        delivery                         no
17   drive_through                        yes
18            name                      Sonic
19            city                     Spring
20     housenumber                

## Most common amenities 

In [40]:
# Note this query only accounts for node tags 
query = ''' select value, count(*) as count from nodetags 
where key = 'amenity' 
group by value
order by count desc
'''
cursor.execute(query)
rows = cursor.fetchall()
df = pd.DataFrame(rows)
print df

                         0    1
0         place_of_worship  193
1                fast_food   89
2                 fountain   89
3               restaurant   81
4                   school   57
5                    bench   33
6             fire_station   24
7                     fuel   23
8                     bank   20
9                 pharmacy   20
10                    cafe   19
11                 toilets   12
12                  police   11
13          drinking_water    9
14                     atm    8
15                 dentist    8
16                 parking    8
17                     bar    7
18                  clinic    6
19                post_box    5
20                     pub    5
21            waste_basket    5
22          bicycle_rental    4
23                 doctors    4
24                 library    4
25  bicycle_repair_station    3
26                car_wash    3
27              grave_yard    3
28             arts_centre    2
29        charging_station    2
30      

In [41]:
# Note this query only accounts for way tags 
query = ''' select value, count(*) as count from waytags 
where key = 'amenity' 
group by value
order by count desc
'''

cursor.execute(query)
rows = cursor.fetchall()
df = pd.DataFrame(rows)
print df

                   0    1
0            parking  411
1             school   94
2          fast_food   45
3               fuel   37
4   place_of_worship   37
5         restaurant   27
6               bank   12
7            shelter   11
8           car_wash   10
9           pharmacy   10
10              cafe    7
11          hospital    7
12          fountain    6
13        grave_yard    6
14           toilets    6
15               bar    4
16      fire_station    4
17           library    4
18            cinema    3
19   bicycle_parking    2
20            clinic    2
21  community_centre    2
22         nightclub    2
23       post_office    2
24            prison    2
25     swimming_pool    2
26        university    2
27       bus_station    1
28        car_rental    1
29           college    1
30           embassy    1
31     parking_space    1
32   public_building    1
33          shopping    1
34          townhall    1
35        veterinary    1


## Most common cities 

In [47]:
query = ''' select key, value, count(*) 
from nodetags 
where key = 'city'
group by value
order by count(*) desc
'''
cursor.execute(query)
rows = cursor.fetchall()
df = pd.DataFrame(rows)
print df

       0               1    2
0   city         Houston  177
1   city        Bellaire   25
2   city        Kingwood   19
3   city        Seabrook    7
4   city          Spring    7
5   city   Missouri City    5
6   city         Cypress    4
7   city     Houston, TX    4
8   city         Tomball    4
9   city      Sugar Land    3
10  city   The Woodlands    3
11  city            Katy    2
12  city        Richmond    2
13  city         Webster    2
14  city         Baytown    1
15  city          Crosby    1
16  city     Friendswood    1
17  city        Fulshear    1
18  city         HOUSTON    1
19  city       Hempstead    1
20  city          Humble    1
21  city  Jersey Village    1
22  city        Katy, TX    1
23  city    Kingwood, TX    1
24  city        La Porte    1
25  city     League City    1
26  city       New Caney    1
27  city    Pasadena, TX    1
28  city        Pearland    1
29  city          Porter    1
30  city        Stafford    1
31  city          Waller    1
32  city  

## Most common leisure buildings

In [43]:
# What are leisure buildings?
query = ''' select *, count(*) as count
from (select * from waytags union all select * from nodetags) as subq
group by value
having key = 'leisure'
order by count desc
limit 40 
'''
cursor.execute(query)
rows = cursor.fetchall()
df = pd.DataFrame(rows)
print df

             0        1                 2        3    4
0   5127141025  leisure             pitch  regular  248
1   5077530823  leisure     swimming_pool  regular  148
2   5261519506  leisure              park  regular  112
3   5568038456  leisure        playground  regular   98
4   4326460364  leisure            garden  regular   19
5    246392936  leisure       golf_course  regular   14
6    579668112  leisure              pool  regular   14
7   4286971468  leisure     sports_centre  regular   12
8   5385232658  leisure      picnic_table  regular   10
9    374906618  leisure           stadium  regular    5
10  5256216786  leisure    fitness_centre  regular    4
11  3717930668  leisure           slipway  regular    3
12   579765111  leisure          dog_park  regular    2
13   465243944  leisure            common  regular    1
14   260566717  leisure  disc_golf_course  regular    1
15    86330818  leisure            marina  regular    1
16   403909954  leisure              walk  regul

# Additional Ideas 

Previously we've observed that node tags and way tags tend to be similar and hold similar pieces of information. It seems that some information may be split across the two tables such that if you were only querying from one, your result may be incomplete and results thus misleading. 

For example, searching for restaurants in an area by querying nodetags may lead you to overlook a choice if additional, unique restaurants were held in the waytags table instead. You may end up missing out on your favorite restaurant. 

To remedy this, you may try to fix the problem at data input, or combine both tables for your queries. 

At data input, additional instructions on creating data for organization can be provided to those contributing to OSM's data. Cycling through to rearrange data from one table to another would be long, tedious, and difficult. Depending on the query, combining tags from both ways and nodes should be sufficient to gather relevant data in the same place. 

Additionally, for further data analysis, it would be more beneficial to read through the possible values for tags first, which should be provided on the OSM wiki. 

In [ ]:
#Create a view that combines way and node tags 
query = ''' create view alltags as select * from nodetags union all select * from waytags '''
cursor.execute(query)
rows = cursor.fetchall()
df = pd.DataFrame(rows)
print df

In [45]:
# A quick query to make sure everything works accordingly 
query = ''' select * from alltags'''
cursor.execute(query)
rows = cursor.fetchall()
df = pd.DataFrame(rows)
print df

                0                1                                     2  \
0       151335252              ele                                    26   
1       151335252       gnis:Class                       Populated Place   
2       151335252      gnis:County                             Fort Bend   
3       151335252  gnis:County_num                                   157   
4       151335252    gnis:ST_alpha                                    TX   
5       151335252      gnis:ST_num                                    48   
6       151335252               id                               1347777   
7       151335252      import_uuid  bb7269ee-502a-5391-8056-e3ce0e66489c   
8       151335252            is_in           Fort Bend,Texas,Tex.,TX,USA   
9       151335252             name                              Stafford   
10      151335252            place                                hamlet   
11      151341161              ele                                    20   
12      1513

In [46]:
# What are the most prevalent restaurants in the area? 
# Note, this query draws from alltags 
query = ''' select names.name, value, count (*) as count 
from 
alltags, 
(SELECT distinct(id) as restid FROM alltags WHERE value= 'restaurant' or value= 'fast_food') 
as rest, 
(select value as name, id as nameid from alltags where key = 'name') as names 

where alltags.id = rest.restid 
and alltags.id = names.nameid 
and alltags.key = 'cuisine' 

group by name 
order by count desc 
'''
cursor.execute(query)
rows = cursor.fetchall()
df = pd.DataFrame(rows)
print df

                                            0                1   2
0                                 Whataburger           burger  16
1                                      Subway         sandwich  12
2                                  McDonald's           burger   8
3                                     Wendy's           burger   8
4                                 Chick-fil-A          chicken   6
5                             Jack in the Box           burger   6
6                                 Burger King           burger   5
7                               Panda Express          chinese   4
8                           Schlotzsky's Deli         sandwich   4
9                                       Sonic           burger   3
10                                Taco Cabana          mexican   3
11                                   Chipotle          mexican   2
12                            Jack In The Box           burger   2
13                                Jamba Juice           drinks

# Conclusion

Upon assessment, it seems that for data collected by humans, it is fairly consistent and thorough (at least more than expected). While this project was challenging, the OSM data seems like a valuable resource for compiling data and practicing data manipulation. 

Most of my frustrations actually occured with preparing the database and creating the database from the csv files. 

If our purpose was to clean the OSM data, then I would likely approach this differently in the future. To reiterate a point from earlier, I thought it better to correct mistakes rather than create a standardization with manmade (ish) data. Instead of standardizing something in one subset of data which may be different in another (depending on the auditor's preference i.e. highway vs hwy), it may be better to compile the various ways contributors may add a data point (Road vs Rd vs Rd.) and group them under the same category when analyzing the data

More familiarization is likely needed with the OSM standards in terms of data possibilities (tags) and organization for other projects. However, many of the top contributors can likely share their code with contributors in other areas in order to speed up the process of preparing and cleaning. 

It seems the challenges faced with this dataset had numerous ways (ha) of being approached, and it would be interesting to see how the top contributors structure their bots to clean the data.  


# Resources

Relevent resources utilized for major roadblocks that occured. 

https://discussions.udacity.com/t/creating-db-file-from-csv-files-with-non-ascii-unicode-characters/174958/7
https://gist.github.com/carlward/54ec1c91b62a5f911c42#file-sample_project-md


In [ ]:
# Udacity code for creating a sample file
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import xml.etree.ElementTree as ET  # Use cElementTree or lxml if too slow

OSM_FILE = "some_osm.osm"  # Replace this with your osm file
SAMPLE_FILE = "sample.osm"

k = 10 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag
    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')
    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))
    output.write('</osm>')